<a href="https://colab.research.google.com/github/ManelSoengas/DL_Task2/blob/main/DL_BETO_Task2_es_24_8_23.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers[torch]
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 78.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 53.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 24.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from sklearn.utils import shuffle
test_data = pd.read_csv('/content/EXIST2021_test_labeled.tsv', sep='\t', usecols=['text', 'language', 'task2']).loc[lambda df: df['language'] == 'es']
#train_data = pd.read_csv('/content/EXIST2021_training.tsv', sep='\t', usecols=['text', 'language', 'task2']).loc[lambda df: df['language'] == 'es']
train_data = pd.read_csv('/content/EXIST2021_training.tsv', sep='\t', usecols=['text', 'language', 'task2']).loc[lambda df: df['language'] =='es']
train_data_es = pd.read_csv('/content/train_data_translated_from_en_to_es.csv', sep=',', usecols=['text', 'language', 'task2']).loc[lambda df: df['language'] == 'en']
#swapp = pd.read_csv('/content/swapp_EXIST2021_training.tsv', sep='\t', usecols=['text', 'language', 'task1']).loc[lambda df: df['language'] == 'es']

In [ ]:
shape_train = train_data.shape
print(shape_train)

(3541, 3)


In [ ]:
import pandas as pd
from sklearn.utils import shuffle
test_data = test_data.loc[(test_data['task2'] != 'non-sexist') & (test_data['language'] == 'es')]
train_data = train_data.loc[(train_data['task2'] != 'non-sexist') & (train_data['language'] == 'es')]
train_data_es = train_data_es.loc[(train_data_es['task2'] != 'non-sexist') & (train_data_es['language'] == 'en')]
train_data = shuffle(train_data, random_state=42)

In [ ]:
shape_train = train_data.shape
print(shape_train)

(1741, 3)


In [ ]:
shape_train_es = train_data_es.shape
print(shape_train_es)

(1636, 3)


In [ ]:
# Concatenar train_data y train_data_en en un único DataFrame llamado train_data
train_data = pd.concat([train_data, train_data_es], ignore_index=True)
train_data = shuffle(train_data, random_state=42)

# Verificar el resultado
print(train_data.head())

     language                                               text  \
3263       en  Si "centramos todas las experiencias", no se c...   
144        es  me he dado cuenta q las mujeres tauro son fabu...   
1231       es  no voy a superar la vez que soñé que yeojin me...   
3001       en                  @CalumTheNBHDSOS lo que sea zorra   
1052       es  @aleperaltamerlo Lo raro es que no salió a mat...   

                             task2  
3263        ideological-inequality  
144                objectification  
1231        ideological-inequality  
3001        stereotyping-dominance  
1052  misogyny-non-sexual-violence  


In [ ]:
columna_clase_objetivo = "task2"

df = train_data

# Contar la cantidad de categorías únicas en la columna de la clase objetivo
cantidad_categorias = df[columna_clase_objetivo].nunique()

# Contar la cantidad de muestras para cada categoría de la clase objetivo
muestras_por_categoria = df[columna_clase_objetivo].value_counts()

# Imprimir los resultados
print("Cantidad de categorías:", cantidad_categorias)
print("Muestras por categoría:")
print(muestras_por_categoria)
df.shape[0]

Cantidad de categorías: 5
Muestras por categoría:
ideological-inequality          866
stereotyping-dominance          809
misogyny-non-sexual-violence    685
sexual-violence                 517
objectification                 500
Name: task2, dtype: int64


3377

#**PREPROCESADO**


---



In [ ]:
import re

def anonimize_text(text):
    # Anonimizar nombres de usuario
    text = re.sub(r'@[\w_]+', '@user', text)

    # Anonimizar hashtags
    text = re.sub(r'#\w+', '#hashtag', text)

    # Anonimizar enlaces
    text = re.sub(r'https?://\S+', 'link', text)

    return text

train_data['text'] = train_data['text'].apply(anonimize_text)
test_data['text'] = test_data['text'].apply(anonimize_text)

In [ ]:
# Convertir a minúsculas

train_data.text = train_data.text.str.lower()
train_data.head(10)
test_data.text = test_data.text.str.lower()
test_data.head(10)

,language,text,task1
0,en,pennsylvania state rep horrifies with opening ...,non-sexist
1,en,"@user he sounds like as ass, and very condesce...",non-sexist
2,en,"@user @user lol! ""this behavior of not letting...",sexist
3,en,@user @user rights?i mean yeah most women espe...,sexist
4,en,the jack manifold appreciation i’m seeing is o...,non-sexist
5,en,@user why isn't this character taking a lie de...,non-sexist
6,en,@user i can’t help but torture myself reading ...,non-sexist
7,en,@user @user @user stop regarding women as anim...,sexist
8,en,"yeah, it is rough, but not for women. married ...",sexist
9,en,although the christian denial isn't emphasized...,non-sexist


In [ ]:
# Eliminar caráctres especiales, #:)(/\='] usados en emojis y hashtags

train_data.text = train_data.text.str.replace("[^a-zA-ZñÑ#]", " ")
train_data.head(10)
test_data.text = test_data.text.str.replace("[^a-zA-ZñÑ#]", " ")
test_data.head(10)

<ipython-input-7-091c5808df50>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  train_data.text = train_data.text.str.replace("[^a-zA-ZñÑ#]", " ")
<ipython-input-7-091c5808df50>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  test_data.text = test_data.text.str.replace("[^a-zA-ZñÑ#]", " ")


,language,text,task1
0,en,pennsylvania state rep horrifies with opening ...,non-sexist
1,en,user he sounds like as ass and very condesce...,non-sexist
2,en,user user lol this behavior of not letting...,sexist
3,en,user user rights i mean yeah most women espe...,sexist
4,en,the jack manifold appreciation i m seeing is o...,non-sexist
5,en,user why isn t this character taking a lie de...,non-sexist
6,en,user i can t help but torture myself reading ...,non-sexist
7,en,user user user stop regarding women as anim...,sexist
8,en,yeah it is rough but not for women married ...,sexist
9,en,although the christian denial isn t emphasized...,non-sexist




---



In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
# Crear una instancia de LabelEncoder
SEED = 4
le = LabelEncoder()
le.fit(train_data['task2'])

# Preprocess the dataset
train_texts, val_texts, train_labels, val_labels = train_test_split(list(train_data['text']), list(le.transform(train_data['task2'])),
                                                                    test_size=.2, random_state=SEED, shuffle=True)

In [ ]:
train_data.head(10)

,language,task2,text
712,es,non-sexist,janis joplin la perla rebelde que cambio el pa...
279,es,non-sexist,diga lo que diga una amiga de verdad hubiera d...
1456,es,non-sexist,me quiero follar al osito bimbo sus deliciosas...
3505,es,objectification,perra sensacion solo dime que quieres que nos ...
2194,es,sexual-violence,user user user user cierto y habla de lo que l...
2080,es,sexual-violence,user user aqui le explico en que partes del pe...
3209,es,misogyny-non-sexual-violence,me voy a pegar un tiro pensando en quien es el...
1788,es,non-sexist,soy super dino no me hablen chica os alejense ...
478,es,non-sexist,no se merece mi poder divino mejor no lo avivo...
1071,es,non-sexist,user camisa de manga corta y corbatina estrech...


In [ ]:
# Preprocess the dataset
test_texts = list(test_data['text'])

test_labels = le.transform(test_data['task2'])


In [ ]:
print(len(train_texts))
print(len(val_texts))
print(len(test_texts))

2701
676
1123


In [ ]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

# Tokenize the data
tokenizer = AutoTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-cased")
# Define the model
model = "dccuchile/bert-base-spanish-wwm-cased"



In [ ]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

# Tokenize the data
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
# Define the model
model = "bert-base-multilingual-cased"

In [ ]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=130)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=130)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=130)


In [ ]:
train_encodings['labels'] = train_labels
val_encodings['labels'] = val_labels
test_encodings['labels'] = test_labels

In [ ]:
batch_size = 16
num_epochs = 6

In [ ]:
!pip install datasets

from transformers import DataCollatorWithPadding
from datasets import Dataset

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

tf_train_dataset = Dataset.from_dict(train_encodings).to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=batch_size,
)

tf_validation_dataset = Dataset.from_dict(val_encodings).to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=batch_size,
)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.5 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/datasets/arrow_dataset.py:400: FutureWarning: The output of `to_tf_dataset` will change when a passing single element list for `labels` or `columns` in the next datasets version. To return a tuple structure rather than dict, pass a single string.
Old behaviour: columns=['a'], labels=['labels'] -> (tf.Tensor, tf.Tensor)  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor)  
New behaviour: columns=['a'],labels=['labels'] -> ({'a': tf.Tensor}, {'labels': tf.Tensor})  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor) 
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [ ]:
from tensorflow.keras.optimizers.schedules import PolynomialDecay
from tensorflow.keras.optimizers import Adam, AdamW
from tensorflow.keras.losses import SparseCategoricalCrossentropy

In [ ]:
num_train_steps = len(tf_train_dataset) * num_epochs
lr_scheduler = PolynomialDecay(
    initial_learning_rate=5e-5, end_learning_rate=0.0, decay_steps=num_train_steps
)

opt = AdamW(learning_rate=lr_scheduler)
loss=SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
model = TFAutoModelForSequenceClassification.from_pretrained(model, num_labels=5,
                                                             attention_probs_dropout_prob=0.1,
                                                             hidden_dropout_prob=0.1,
                                                             from_pt=True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias', 'classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

class_weights = compute_class_weight('balanced', classes=np.unique(train_labels), y=train_labels)
class_weights_dict = {i: class_weights[i] for i in range(len(class_weights))}


In [ ]:
for class_idx, weight in class_weights_dict.items():
    print(f'Class {class_idx}: Weight = {weight:.4f}')


Class 0: Weight = 0.7818
Class 1: Weight = 0.9840
Class 2: Weight = 1.4068
Class 3: Weight = 1.2862
Class 4: Weight = 0.8222


In [ ]:


model.compile(
    optimizer=opt,
    loss=loss,
    metrics=["accuracy"]
)


In [ ]:
model.fit(
    tf_train_dataset,
    validation_data=tf_validation_dataset,
    epochs=num_epochs,
    class_weight=class_weights_dict
)

Epoch 1/6
85/85 [==============================] - 135s 900ms/step - loss: 1.2154 - accuracy: 0.5087 - val_loss: 0.8981 - val_accuracy: 0.6820
Epoch 2/6
85/85 [==============================] - 76s 888ms/step - loss: 0.7005 - accuracy: 0.7575 - val_loss: 0.9203 - val_accuracy: 0.6672
Epoch 3/6
85/85 [==============================] - 76s 900ms/step - loss: 0.3463 - accuracy: 0.8874 - val_loss: 1.0849 - val_accuracy: 0.6657
Epoch 4/6
85/85 [==============================] - 76s 900ms/step - loss: 0.1309 - accuracy: 0.9626 - val_loss: 1.1702 - val_accuracy: 0.6834
Epoch 5/6
85/85 [==============================] - 76s 899ms/step - loss: 0.0462 - accuracy: 0.9911 - val_loss: 1.2658 - val_accuracy: 0.6775
Epoch 6/6
85/85 [==============================] - 76s 900ms/step - loss: 0.0227 - accuracy: 0.9974 - val_loss: 1.2971 - val_accuracy: 0.6834


In [ ]:
model.fit(
    tf_train_dataset,
    validation_data=tf_validation_dataset,
    epochs=num_epochs
)

Epoch 1/6
169/169 [==============================] - 164s 573ms/step - loss: 1.1609 - accuracy: 0.5461 - val_loss: 0.9452 - val_accuracy: 0.6538
Epoch 2/6
169/169 [==============================] - 87s 515ms/step - loss: 0.6778 - accuracy: 0.7675 - val_loss: 0.8771 - val_accuracy: 0.6805
Epoch 3/6
169/169 [==============================] - 84s 499ms/step - loss: 0.3474 - accuracy: 0.8912 - val_loss: 1.0729 - val_accuracy: 0.6553
Epoch 4/6
169/169 [==============================] - 83s 490ms/step - loss: 0.1515 - accuracy: 0.9574 - val_loss: 1.2235 - val_accuracy: 0.6583
Epoch 5/6
169/169 [==============================] - 83s 492ms/step - loss: 0.0593 - accuracy: 0.9885 - val_loss: 1.2679 - val_accuracy: 0.6598
Epoch 6/6
169/169 [==============================] - 82s 487ms/step - loss: 0.0310 - accuracy: 0.9959 - val_loss: 1.3045 - val_accuracy: 0.6657


In [ ]:
tf_test_dataset = Dataset.from_dict(test_encodings).to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=False,  # No necesitas barajar para predicción
    collate_fn=data_collator,
    batch_size=batch_size,
)


In [ ]:
import numpy as np
preds = model.predict(tf_test_dataset)["logits"]
class_preds = np.argmax(preds, axis=1)

71/71 [==============================] - 15s 149ms/step


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(class_preds, test_encodings['labels']))

              precision    recall  f1-score   support

           0       0.83      0.72      0.77       332
           1       0.65      0.59      0.62       284
           2       0.52      0.68      0.59       132
           3       0.71      0.77      0.74       185
           4       0.62      0.66      0.64       190

    accuracy                           0.68      1123
   macro avg       0.66      0.68      0.67      1123
weighted avg       0.69      0.68      0.68      1123



# **Resultados**

# **Se utiliza el modelo BETO para tuits en español. Se utilizan los datos originales más un aumento de datos que provienen de la traducción de los datos en inglés al español.**
batch_size = 16
num_epochs = 4

#Train


```
Epoch 1/4
349/349 [==============================] - 238s 522ms/step - loss: 1.1413 - accuracy: 0.5942 - val_loss: 0.9411 - val_accuracy: 0.6819
Epoch 2/4
349/349 [==============================] - 174s 499ms/step - loss: 0.7248 - accuracy: 0.7422 - val_loss: 0.8973 - val_accuracy: 0.6784
Epoch 3/4
349/349 [==============================] - 165s 472ms/step - loss: 0.3440 - accuracy: 0.8843 - val_loss: 1.0380 - val_accuracy: 0.6834
Epoch 4/4
349/349 [==============================] - 165s 473ms/step - loss: 0.1232 - accuracy: 0.9685 - val_loss: 1.1750 - val_accuracy: 0.6784
```

#Evaluación


```
precision    recall  f1-score   support

           0       0.68      0.69      0.69       285
           1       0.49      0.49      0.49       255
           2       0.83      0.74      0.78      1163
           3       0.34      0.49      0.40       123
           4       0.50      0.71      0.59       142
           5       0.50      0.53      0.51       192

    accuracy                           0.67      2160
   macro avg       0.56      0.61      0.58      2160
weighted avg       0.69      0.67      0.68      2160
```



# **Se utiliza el modelo BETO para tuits en español. Se utilizan los datos originales más un aumento de datos que provienen de la traducción de los datos en inglés al español. batch_size = 32 num_epochs = 8**

# Train


```
Epoch 1/8
175/175 [==============================] - 221s 945ms/step - loss: 1.1562 - accuracy: 0.5884 - val_loss: 0.9143 - val_accuracy: 0.6755
Epoch 2/8
175/175 [==============================] - 155s 883ms/step - loss: 0.7326 - accuracy: 0.7423 - val_loss: 0.9471 - val_accuracy: 0.6827
Epoch 3/8
175/175 [==============================] - 163s 934ms/step - loss: 0.3705 - accuracy: 0.8780 - val_loss: 1.1259 - val_accuracy: 0.6683
Epoch 4/8
175/175 [==============================] - 156s 891ms/step - loss: 0.1605 - accuracy: 0.9514 - val_loss: 1.2585 - val_accuracy: 0.6719
Epoch 5/8
175/175 [==============================] - 162s 928ms/step - loss: 0.0664 - accuracy: 0.9835 - val_loss: 1.4735 - val_accuracy: 0.6562
Epoch 6/8
175/175 [==============================] - 154s 880ms/step - loss: 0.0282 - accuracy: 0.9943 - val_loss: 1.5172 - val_accuracy: 0.6755
Epoch 7/8
175/175 [==============================] - 155s 889ms/step - loss: 0.0169 - accuracy: 0.9973 - val_loss: 1.6009 - val_accuracy: 0.6705
Epoch 8/8
175/175 [==============================] - 155s 884ms/step - loss: 0.0102 - accuracy: 0.9982 - val_loss: 1.6118 - val_accuracy: 0.6712


```
# Evaluación


```
precision    recall  f1-score   support

           0       0.64      0.67      0.65       278
           1       0.53      0.47      0.50       286
           2       0.83      0.73      0.77      1177
           3       0.32      0.54      0.40       104
           4       0.46      0.72      0.56       129
           5       0.53      0.58      0.56       186

    accuracy                           0.66      2160
   macro avg       0.55      0.62      0.57      2160
weighted avg       0.69      0.66      0.67      2160
```



# **Se utiliza el modelo BETO para tuits en español. Se utilizan los datos originales ( solo categorias sexistas ) sin un aumento de datos que provienen de la traducción de los datos en inglés al español. batch_size = 16 num_epochs = 4**

#Train


```
Epoch 1/4
87/87 [==============================] - 117s 596ms/step - loss: 1.2471 - accuracy: 0.4864 - val_loss: 0.8954 - val_accuracy: 0.6877
Epoch 2/4
87/87 [==============================] - 46s 525ms/step - loss: 0.7032 - accuracy: 0.7450 - val_loss: 0.8933 - val_accuracy: 0.6762
Epoch 3/4
87/87 [==============================] - 42s 481ms/step - loss: 0.3259 - accuracy: 0.9023 - val_loss: 1.0039 - val_accuracy: 0.6648
Epoch 4/4
87/87 [==============================] - 44s 501ms/step - loss: 0.1387 - accuracy: 0.9677 - val_loss: 1.0647 - val_accuracy: 0.6705
```

#Evaluación


```
precision    recall  f1-score   support

           0       0.86      0.70      0.77       353
           1       0.58      0.66      0.62       229
           2       0.60      0.65      0.62       162
           3       0.69      0.85      0.76       163
           4       0.68      0.64      0.66       216

    accuracy                           0.69      1123
   macro avg       0.68      0.70      0.69      1123
weighted avg       0.71      0.69      0.70      1123

```



# **Se utiliza el modelo BETO para tuits en español. Se utilizan los datos originales ( solo categorias sexistas ) con un aumento de datos que provienen de la traducción de los datos en inglés al español. batch_size = 16 num_epochs = 4**

#Train


```
Epoch 1/4
169/169 [==============================] - 175s 583ms/step - loss: 1.1415 - accuracy: 0.5461 - val_loss: 0.9499 - val_accuracy: 0.6538
Epoch 2/4
169/169 [==============================] - 84s 500ms/step - loss: 0.6608 - accuracy: 0.7664 - val_loss: 0.8967 - val_accuracy: 0.6790
Epoch 3/4
169/169 [==============================] - 86s 508ms/step - loss: 0.2749 - accuracy: 0.9148 - val_loss: 1.0524 - val_accuracy: 0.6775
Epoch 4/4
169/169 [==============================] - 85s 502ms/step - loss: 0.1017 - accuracy: 0.9759 - val_loss: 1.1500 - val_accuracy: 0.6834
<keras.callbacks.History at 0x798220277a30>
```

#Evaluación



```
precision    recall  f1-score   support

           0       0.83      0.69      0.75       346
           1       0.60      0.57      0.58       271
           2       0.49      0.66      0.56       128
           3       0.67      0.73      0.70       185
           4       0.62      0.65      0.64       193

    accuracy                           0.66      1123
   macro avg       0.64      0.66      0.65      1123
weighted avg       0.67      0.66      0.66      1123

```

